# Bachata Dance Analysis Demo

This notebook demonstrates the complete Bachata Video Professor pipeline for extracting dance combinations from YouTube videos.

## Overview

The pipeline performs the following steps:
1. **Video Loading**: Downloads or loads video from YouTube/local file
2. **Pose Detection**: Uses MediaPipe to detect human poses on CPU
3. **Multi-person Tracking**: Tracks all people and selects primary dancing couple
4. **Feature Extraction**: Extracts motion features for each frame
5. **Segmentation**: Identifies individual dance combinations
6. **Role Identification**: Determines leader/follower roles
7. **Output Generation**: Creates JSON, text summary, and annotated video

## Setup

First, let's install the required packages and import the necessary modules.

In [ ]:
# Install required packages (uncomment if running in Colab/Kaggle)
# !pip install mediapipe opencv-python numpy scipy tqdm pydantic yt-dlp matplotlib scikit-learn

import sys
import os
from pathlib import Path

# Add the src directory to Python path
if os.path.exists('../src'):
    sys.path.append('../src')
elif os.path.exists('./src'):
    sys.path.append('./src')

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Video, HTML

from bachata_analyzer import BachataAnalyzer, AnalysisConfig
from bachata_analyzer.models import AnalysisResult, DanceSegment

## Configuration

Let's set up the analysis configuration. You can adjust these parameters based on your needs:

In [ ]:
# Configure analysis parameters
config = AnalysisConfig(
    fps=12,                    # Frames per second for analysis
    max_width=1280,            # Maximum video width (downscale for speed)
    min_segment_sec=4.0,       # Minimum segment length in seconds
    pose_confidence=0.5,       # MediaPipe pose confidence threshold
    tracking_confidence=0.5,   # MediaPipe tracking confidence threshold
    create_video=True,         # Create annotated video output
    use_temporal_smoothing=True,  # Apply temporal smoothing to keypoints
    smoothing_window=5,        # Window size for temporal smoothing
    change_point_sensitivity=0.8,  # Sensitivity for detecting combo changes
    pause_threshold=0.3        # Threshold for detecting pauses
)

print("Configuration:")
print(f"  FPS: {config.fps}")
print(f"  Max Width: {config.max_width}px")
print(f"  Min Segment Length: {config.min_segment_sec}s")
print(f"  Create Video: {config.create_video}")

## Example Videos

Here are the test videos we'll analyze:

In [ ]:
# Example YouTube URLs
example_videos = {
    "bachata_demo_1": "https://youtu.be/4V7EccGsSUI?si=m-hxgRejZcg-b1nD",
    "bachata_demo_2": "https://youtu.be/OIEpCz8Q97A?si=k08QOEL5rkqALeDH",
    "bachata_demo_3": "https://youtu.be/6MqwgPIiQaQ?si=LE9Tp8s-1wQ-OhBX"
}

print("Available example videos:")
for name, url in example_videos.items():
    print(f"  {name}: {url}")

## Analysis Example

Let's analyze one of the example videos. This will take a few minutes depending on your CPU and video length.

In [ ]:
# Select video to analyze
video_name = "bachata_demo_1"
video_url = example_videos[video_name]
output_dir = Path(f"./output/{video_name}")

print(f"Analyzing: {video_url}")
print(f"Output directory: {output_dir}")
print("\nThis may take 5-15 minutes depending on your CPU...")

# Run analysis
with BachataAnalyzer(config) as analyzer:
    result = analyzer.analyze(video_url, output_dir)

print(f"\nAnalysis complete!")
print(f"Found {len(result.segments)} dance combinations")
print(f"Total dance time: {result.get_total_dance_time():.1f}s")
print(f"Dance coverage: {(result.get_total_dance_time() / result.duration_sec) * 100:.1f}%")

## Results Summary

Let's examine the analysis results:

In [ ]:
# Load and display results
import json

# Load JSON results
json_path = output_dir / "segments.json"
with open(json_path, 'r') as f:
    results_data = json.load(f)

print(f"Video ID: {results_data['video_id']}")
print(f"Duration: {results_data['duration_sec']:.1f} seconds")
print(f"FPS: {results_data['fps']}")
print(f"Total segments: {len(results_data['segments'])}")

# Display first few segments
print("\nFirst 5 segments:")
for i, segment in enumerate(results_data['segments'][:5]):
    print(f"\n  {segment['tentative_name']}: {segment['start_sec']:.1f}s - {segment['end_sec']:.1f}s")
    print(f"    Duration: {segment['end_sec'] - segment['start_sec']:.1f}s")
    print(f"    Leader: {segment['leader_name']}, Follower: {segment['follower_name']}")
    print(f"    Avg Speed: {segment['features']['avg_speed']:.4f}")
    print(f"    Turns: {segment['features']['turns']}, Dip: {segment['features']['dip']}")

## Feature Analysis

Let's visualize some of the extracted features:

In [ ]:
# Extract features for visualization
segments = results_data['segments']

# Create feature arrays
speeds = [seg['features']['avg_speed'] for seg in segments]
hand_distances = [seg['features']['hand_distance_avg'] for seg in segments]
step_cadences = [seg['features']['step_cadence'] for seg in segments]
durations = [seg['end_sec'] - seg['start_sec'] for seg in segments]
turns = [seg['features']['turns'] for seg in segments]
dips = [seg['features']['dip'] for seg in segments]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Dance Segment Features', fontsize=16)

# Speed distribution
axes[0, 0].hist(speeds, bins=20, alpha=0.7, color='blue')
axes[0, 0].set_title('Average Speed Distribution')
axes[0, 0].set_xlabel('Speed')
axes[0, 0].set_ylabel('Count')

# Hand distance distribution
axes[0, 1].hist(hand_distances, bins=20, alpha=0.7, color='green')
axes[0, 1].set_title('Hand Distance Distribution')
axes[0, 1].set_xlabel('Hand Distance')
axes[0, 1].set_ylabel('Count')

# Step cadence distribution
axes[0, 2].hist(step_cadences, bins=20, alpha=0.7, color='red')
axes[0, 2].set_title('Step Cadence Distribution')
axes[0, 2].set_xlabel('Step Cadence')
axes[0, 2].set_ylabel('Count')

# Duration distribution
axes[1, 0].hist(durations, bins=20, alpha=0.7, color='purple')
axes[1, 0].set_title('Segment Duration Distribution')
axes[1, 0].set_xlabel('Duration (seconds)')
axes[1, 0].set_ylabel('Count')

# Turns and dips
turn_count = sum(turns)
dip_count = sum(dips)
no_special = len(turns) - turn_count - dip_count

axes[1, 1].bar(['Turns', 'Dips', 'Neither'], [turn_count, dip_count, no_special], 
               color=['orange', 'pink', 'gray'])
axes[1, 1].set_title('Special Moves Count')
axes[1, 1].set_ylabel('Count')

# Timeline visualization
segment_times = [(seg['start_sec'], seg['end_sec']) for seg in segments]
for i, (start, end) in enumerate(segment_times):
    color = 'red' if turns[i] else 'blue' if dips[i] else 'green'
    axes[1, 2].barh(i, end - start, left=start, height=0.8, color=color, alpha=0.7)

axes[1, 2].set_title('Dance Timeline')
axes[1, 2].set_xlabel('Time (seconds)')
axes[1, 2].set_ylabel('Segment')

plt.tight_layout()
plt.show()

# Print statistics
print(f"\nFeature Statistics:")
print(f"  Average speed: {np.mean(speeds):.4f} ± {np.std(speeds):.4f}")
print(f"  Average hand distance: {np.mean(hand_distances):.4f} ± {np.std(hand_distances):.4f}")
print(f"  Average segment duration: {np.mean(durations):.1f} ± {np.std(durations):.1f}s")
print(f"  Segments with turns: {turn_count}/{len(segments)} ({100*turn_count/len(segments):.1f}%)")
print(f"  Segments with dips: {dip_count}/{len(segments)} ({100*dip_count/len(segments):.1f}%)")

## Text Summary

Let's display the generated text summary:

In [ ]:
# Display text summary
summary_path = output_dir / "summary.md"
with open(summary_path, 'r') as f:
    summary_content = f.read()

print(summary_content[:2000])  # Display first 2000 characters
print("... (truncated for display)")

## Annotated Video

If video generation was enabled, let's display the annotated video:

In [ ]:
# Check if annotated video exists
video_path = output_dir / "annotated.mp4"

if video_path.exists():
    print(f"Annotated video available: {video_path}")
    print(f"Video size: {video_path.stat().st_size / (1024*1024):.1f} MB")
    
    # Display video (works in Jupyter/Colab)
    try:
        display(Video(str(video_path), width=640, height=480))
    except:
        print("Video display not available in this environment")
        print(f"You can download the video from: {video_path}")
else:
    print("Annotated video not found. Video generation may have been disabled or failed.")

## Performance Analysis

Let's analyze the performance and accuracy of our segmentation:

In [ ]:
# Performance metrics
total_segments = len(segments)
total_dance_time = result.get_total_dance_time()
video_duration = result.duration_sec
dance_coverage = (total_dance_time / video_duration) * 100
avg_segment_length = total_dance_time / total_segments

print("Performance Analysis:")
print(f"  Total segments detected: {total_segments}")
print(f"  Total dance time: {total_dance_time:.1f}s")
print(f"  Video duration: {video_duration:.1f}s")
print(f"  Dance coverage: {dance_coverage:.1f}%")
print(f"  Average segment length: {avg_segment_length:.1f}s")

# Segment quality metrics
quality_scores = []
for seg in segments:
    # Simple quality score based on movement and features
    score = seg['features']['avg_speed'] * 10
    if seg['features']['turns']:
        score += 2
    if seg['features']['dip']:
        score += 3
    quality_scores.append(score)

print(f"\nSegment Quality:")
print(f"  Average quality score: {np.mean(quality_scores):.2f} ± {np.std(quality_scores):.2f}")
print(f"  Highest quality segment: {np.argmax(quality_scores) + 1}")
print(f"  Lowest quality segment: {np.argmin(quality_scores) + 1}")

# Find most interesting segments
fastest_segments = sorted(range(len(speeds)), key=lambda i: speeds[i], reverse=True)[:3]
slowest_segments = sorted(range(len(speeds)), key=lambda i: speeds[i])[:3]

print(f"\nMost Dynamic Segments:")
for i in fastest_segments:
    seg = segments[i]
    print(f"  {seg['tentative_name']}: {seg['start_sec']:.1f}s-{seg['end_sec']:.1f}s (speed: {speeds[i]:.4f})")

print(f"\nSlowest Segments:")
for i in slowest_segments:
    seg = segments[i]
    print(f"  {seg['tentative_name']}: {seg['start_sec']:.1f}s-{seg['end_sec']:.1f}s (speed: {speeds[i]:.4f})")

## Custom Analysis

You can analyze other videos by modifying the URL below:

In [ ]:
# Analyze a custom video (uncomment and modify)
# custom_url = "https://www.youtube.com/watch?v=YOUR_VIDEO_ID"
# custom_output = Path("./output/custom_analysis")

# with BachataAnalyzer(config) as analyzer:
#     custom_result = analyzer.analyze(custom_url, custom_output)
#     print(f"Found {len(custom_result.segments)} combinations in custom video")

## Configuration Tuning

Experiment with different configurations to see how they affect the results:

In [ ]:
# Test different configurations
configs_to_test = [
    {"name": "Fast Processing", "fps": 8, "max_width": 720},
    {"name": "High Quality", "fps": 15, "max_width": 1280},
    {"name": "Sensitive Segmentation", "min_segment_sec": 2.0, "change_point_sensitivity": 1.2},
    {"name": "Conservative Segmentation", "min_segment_sec": 6.0, "change_point_sensitivity": 0.5}
]

# Uncomment to test different configs
# for config_params in configs_to_test:
#     print(f"\nTesting: {config_params['name']}")
#     test_config = AnalysisConfig(**{k: v for k, v in config_params.items() if k != 'name'})
#     
#     test_output = Path(f"./output/test_{config_params['name'].lower().replace(' ', '_')}")
#     
#     with BachataAnalyzer(test_config) as analyzer:
#         test_result = analyzer.analyze(video_url, test_output)
#         print(f"  Segments found: {len(test_result.segments)}")
#         print(f"  Processing time: (varies by config)")

## Conclusion

This notebook demonstrated the complete Bachata Video Professor pipeline:

### Key Features:
- **CPU-only processing**: Works on standard laptops without GPU
- **Multi-person tracking**: Automatically identifies the primary dancing couple
- **Unsupervised segmentation**: Detects dance combinations without manual labeling
- **Multiple outputs**: JSON data, human-readable summary, and annotated video
- **Configurable parameters**: Adjust speed vs. accuracy tradeoffs

### Usage Tips:
- Lower FPS and resolution for faster processing
- Adjust segment length based on dance style
- Use temporal smoothing for more stable pose tracking
- Experiment with change point sensitivity for different dance styles

### Next Steps:
- Try with different Bachata videos
- Adjust parameters for your specific use case
- Integrate with your own dance analysis workflow
- Extend with custom features or classifiers